## Control del séptimo Eje 

In [2]:
import mujoco
import mujoco.viewer
import time
import numpy as np

model = mujoco.MjModel.from_xml_path("wherehouse_system.xml")
data = mujoco.MjData(model)

# Variables globales
posiciones = [0.0, -1.5,1.5]
idx = 0
tiempo_inicio = time.time()

def inicializar_robot_posicion(config_deseada):
    """
    Inicializa el robot en una configuración específica
    
    Args:
        config_deseada: [linear, shoulder_pan, shoulder_lift, elbow, wrist_1, wrist_2, wrist_3]
    """
    print(f"Inicializando robot en configuración: {config_deseada}")
    
    # Nombres de las articulaciones en orden
    joints = [
        "linear_joint",
        "shoulder_pan_joint",
        "shoulder_lift_joint", 
        "elbow_joint",
        "wrist_1_joint",
        "wrist_2_joint",
        "wrist_3_joint"
    ]
    
    # Establecer posiciones de los joints
    for i, joint_name in enumerate(joints):
        joint_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, joint_name)
        data.qpos[joint_id] = config_deseada[i]
        
        # También establecer control para que se mantenga
        if i < len(data.ctrl):
            data.ctrl[i] = config_deseada[i]
    
    # Actualizar simulación
    mujoco.mj_forward(model, data)
    
    print(f"Robot inicializado")
    
    # Verificar posición del efector final
    efector_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "wrist_3_link")
    pos_efector = data.xpos[efector_id]
    print(f"Posición efector final: {np.round(pos_efector, 3)}")

def imprimir_posicion_efector():
    """Imprime la posición del efector final del robot UR10e"""
    try:
        body_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "wrist_3_link")
        pos = data.xpos[body_id]
        print(f"Efector final: x={pos[0]:.3f}, y={pos[1]:.3f}, z={pos[2]:.3f}")
    except Exception as e:
        print(f"Error: {e}")

def imprimir_posicion_base():
    """Imprime la posición de la base del robot UR10e"""
    try:
        body_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "base")
        pos = data.xpos[body_id]
        print(f"Base robot: x={pos[0]:.3f}, y={pos[1]:.3f}, z={pos[2]:.3f}")
    except Exception as e:
        print(f"Error: {e}")
        
def imprimir_posiciones_articulaciones():
    """Imprime las posiciones de todas las articulaciones del robot"""
    try:
        joints = [
            "linear_joint",
            "shoulder_pan_joint",
            "shoulder_lift_joint", 
            "elbow_joint",
            "wrist_1_joint",
            "wrist_2_joint",
            "wrist_3_joint"
        ]
        
        print("Posiciones articulaciones:")
        for joint_name in joints:
            joint_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, joint_name)
            pos = data.qpos[joint_id]
            print(f"  {joint_name}: {pos:.3f}")
    except Exception as e:
        print(f"Error: {e}")

def ir_a_posicion(pos):
    """Mueve el eje a la posición especificada"""
    data.ctrl[0] = pos

# ===== INICIALIZACIÓN CON CONFIGURACIÓN ESPECÍFICA =====
# Configuración deseada: [linear, shoulder_pan, shoulder_lift, elbow, wrist_1, wrist_2, wrist_3]
configuracion_inicial = [0, 1.76, -1.19, 1.6, -2.2, -1.76, 0]

# Inicializar robot en la configuración deseada
inicializar_robot_posicion(configuracion_inicial)

# Mostrar estado inicial
print("\n Estado inicial del robot:")
imprimir_posiciones_articulaciones()
imprimir_posicion_efector()

# Inicializar variables de control
ir_a_posicion(posiciones[0])

print(f"\n Iniciando simulación...")

with mujoco.viewer.launch_passive(model, data) as viewer:
    while viewer.is_running():
        mujoco.mj_step(model, data)
        
        # Calcular qué posición usar basado en el tiempo transcurrido
        tiempo_transcurrido = time.time() - tiempo_inicio
        posicion_actual = int(tiempo_transcurrido / 6) % len(posiciones)
        
        ir_a_posicion(posiciones[posicion_actual])
        
        # Mostrar cambios de posición
        if posicion_actual != idx:
            idx = posicion_actual
            print(f"Cambiando a posición: {posiciones[idx]:.1f}m")
            # Opcional: imprimir estado cuando cambia
            # imprimir_posicion_efector()
        
        viewer.sync()
        time.sleep(0.01)

Inicializando robot en configuración: [0, 1.76, -1.19, 1.6, -2.2, -1.76, 0]
Robot inicializado
Posición efector final: [ 0.334 -0.82   1.049]

 Estado inicial del robot:
Posiciones articulaciones:
  linear_joint: 0.000
  shoulder_pan_joint: 1.760
  shoulder_lift_joint: -1.190
  elbow_joint: 1.600
  wrist_1_joint: -2.200
  wrist_2_joint: -1.760
  wrist_3_joint: 0.000
Efector final: x=0.334, y=-0.820, z=1.049

 Iniciando simulación...
Cambiando a posición: -1.5m
Cambiando a posición: 1.5m
Cambiando a posición: 0.0m


In [ ]:
# ===== EJEMPLO DE USO =====
if __name__ == "__main__":
    # Cargar modelo
    model = mujoco.MjModel.from_xml_path("wherehouse_system.xml")
    data = mujoco.MjData(model)
    
    print("🎬 EJECUCIÓN DE TRAYECTORIAS EN MUJOCO")
    print("=" * 50)
    
    # Crear trayectoria de ejemplo
    trayectoria_puntos = crear_trayectoria_ejemplo()
    
    print(f"📋 Trayectoria creada con {len(trayectoria_puntos)} puntos:")
    for i, punto in enumerate(trayectoria_puntos):
        print(f"   Punto {i+1}: {np.round(punto, 3)}")
    
    # Opción 1: Ejecución básica
    print(f"\n🎯 OPCIÓN 1: Ejecución básica (saltos discretos)")
    input("Presiona Enter para continuar...")
    ejecutar_trayectoria(model, data, trayectoria_puntos, duracion_total=15.0)
    
    # Opción 2: Ejecución interpolada
    print(f"\n🎯 OPCIÓN 2: Ejecución interpolada (movimiento suave)")
    input("Presiona Enter para continuar...")
    ejecutar_trayectoria_interpolada(model, data, trayectoria_puntos, duracion_total=12.0, interpolar=True)
    
    print(f"\n✅ ¡Ejecución de trayectorias completada!")